In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df = pd.read_csv('../../hao_dataset_test_HI.csv')
df

,doi,in_source,in_amount_mmol,p_source,p_amount_mmol,sol,sol_amount_ml,TOP,TOP_amount_mmol,acid,...,other,other_amount_mmol,total_volume_ml,temp_c,time_min,diameter_nm,abs_nm,emission_nm,date_input,user
0,10.1002/anie.200800281,indium acetate,0.50,phosphorus trichloride,0.50,octadecene,34.0,None,0.000000,stearic acid,...,superhydride,3.0,34.901,250,240.0,3.5,562,618,6/25/2021,Hao
1,10.1002/anie.200800281,indium acetate,0.50,phosphorus trichloride,0.50,octadecene,34.0,None,0.000000,stearic acid,...,superhydride,3.0,34.901,210,60.0,None,473,None,6/25/2021,Hao
2,10.1002/anie.200800281,indium acetate,0.50,phosphorus trichloride,0.50,octadecene,34.0,None,0.000000,stearic acid,...,superhydride,3.0,34.901,230,60.0,None,524,None,6/25/2021,Hao
3,10.1002/anie.200800281,indium acetate,0.50,phosphorus trichloride,0.50,octadecene,34.0,None,0.000000,stearic acid,...,superhydride,3.0,34.901,250,60.0,None,538,None,6/25/2021,Hao
4,10.1002/anie.200800281,indium acetate,0.50,phosphorus trichloride,0.50,octadecene,34.0,None,0.000000,stearic acid,...,superhydride,3.0,34.901,270,60.0,None,576,None,6/25/2021,Hao
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,10.1021/ja309863n,indium myristate,0.30,tris(trimethylgermyl)phosphine,0.15,octadecene,4.0,trioctylphosphine,2.242095,None,...,None,0.0,5.000,210,14.0,None,552,601,6/25/2021,Hao
153,10.1557/jmr.2006.0068,indium trifluoroacetate,0.11,tris(trimethylsilyl)phosphine,0.10,octadecene,7.0,None,0.000000,myristic acid,...,None,0.0,7.000,270,240.0,8.3,532,590,6/25/2021,Hao
154,10.1557/jmr.2006.0068,indium trifluoroacetate,0.11,tris(trimethylsilyl)phosphine,0.10,octadecene,7.0,None,0.000000,myristic acid,...,None,0.0,7.000,270,6.0,None,570,None,6/25/2021,Hao
155,10.1557/jmr.2006.0068,indium trifluoroacetate,0.11,tris(trimethylsilyl)phosphine,0.10,octadecene,7.0,None,0.000000,myristic acid,...,None,0.0,7.000,270,60.0,None,562,None,6/25/2021,Hao


In [3]:
#Checks if there are any columns with no values
df.isna().sum()

doi                  0
in_source            0
in_amount_mmol       0
p_source             0
p_amount_mmol        0
sol                  0
sol_amount_ml        0
TOP                  0
TOP_amount_mmol      0
acid                 0
acid_amount_mmol     0
amine                0
amine_amount_mmol    0
thiol                0
thiol_amount_mmol    0
zinc                 0
zinc_amount_mmol     0
other                0
other_amount_mmol    0
total_volume_ml      0
temp_c               0
time_min             0
diameter_nm          0
abs_nm               0
emission_nm          0
date_input           0
user                 0
dtype: int64

In [4]:
#Separate out initial DataFrame into the input features and output features
df_input = df.drop(columns =['diameter_nm', 'abs_nm', 'emission_nm','doi','user','date_input'], inplace = False, axis = 1)
df_output = df[['diameter_nm', 'abs_nm', 'emission_nm']]

In [5]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [6]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          157 non-null    object 
 1   in_amount_mmol     157 non-null    float64
 2   p_source           157 non-null    object 
 3   p_amount_mmol      157 non-null    float64
 4   sol                157 non-null    object 
 5   sol_amount_ml      157 non-null    float64
 6   TOP                157 non-null    object 
 7   TOP_amount_mmol    157 non-null    float64
 8   acid               157 non-null    object 
 9   acid_amount_mmol   157 non-null    float64
 10  amine              157 non-null    object 
 11  amine_amount_mmol  157 non-null    float64
 12  thiol              157 non-null    object 
 13  thiol_amount_mmol  157 non-null    int64  
 14  zinc               157 non-null    object 
 15  zinc_amount_mmol   157 non-null    float64
 16  other              157 non

In [7]:
df_input['temp_c'] = df_input['temp_c'].astype(float)
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          157 non-null    object 
 1   in_amount_mmol     157 non-null    float64
 2   p_source           157 non-null    object 
 3   p_amount_mmol      157 non-null    float64
 4   sol                157 non-null    object 
 5   sol_amount_ml      157 non-null    float64
 6   TOP                157 non-null    object 
 7   TOP_amount_mmol    157 non-null    float64
 8   acid               157 non-null    object 
 9   acid_amount_mmol   157 non-null    float64
 10  amine              157 non-null    object 
 11  amine_amount_mmol  157 non-null    float64
 12  thiol              157 non-null    object 
 13  thiol_amount_mmol  157 non-null    int64  
 14  zinc               157 non-null    object 
 15  zinc_amount_mmol   157 non-null    float64
 16  other              157 non

In [8]:
#Initializes 2 lists to contain all of the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']

In [9]:
input_cat_cols

['in_source',
 'p_source',
 'sol',
 'TOP',
 'acid',
 'amine',
 'thiol',
 'zinc',
 'other']

In [10]:
#Initializes the ColumnTransformer object, and specifies what it will do with a passed in dataframe
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)
], remainder = 'passthrough')

In [11]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,-0.467859,-0.729457,-0.258929,0.421871,-0.453863,-0.800754,0.0,-0.682991,-0.239089,-0.333650,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
ct.transformers_

[('step1',
  StandardScaler(),
  ['in_amount_mmol',
   'p_amount_mmol',
   'sol_amount_ml',
   'TOP_amount_mmol',
   'acid_amount_mmol',
   'amine_amount_mmol',
   'thiol_amount_mmol',
   'zinc_amount_mmol',
   'other_amount_mmol',
   'total_volume_ml',
   'temp_c',
   'time_min']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['in_source',
   'p_source',
   'sol',
   'TOP',
   'acid',
   'amine',
   'thiol',
   'zinc',
   'other'])]

In [13]:
#Look into ct object to see the column titles
array_of_cat_titles = ct.transformers_[1][1].get_feature_names()
len(array_of_cat_titles) #There are a total of 41 encoded categorical columns

46

In [14]:
#Number of numerical columns in the dataset
len(input_num_cols)

12

In [15]:
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

for j in range(len(array_of_cat_titles)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i+1]: array_of_cat_titles[j]}, inplace = True)
    i = i + 1

In [16]:
df_input_scaled_encoded

,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x7_zinc undecylenate,x8_None,x8_acetic acid,x8_copper bromide,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide
0,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,-0.467859,-0.729457,-0.258929,0.421871,-0.453863,-0.800754,0.0,-0.682991,-0.239089,-0.333650,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x8_copper bromide,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm
0,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.5,562,618
1,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,473,None
2,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,524,None
3,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,538,None
4,-0.205407,-0.376673,1.638502,-0.220165,0.095362,-0.800754,0.0,-0.682991,2.533209,1.502083,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,None,576,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,-0.467859,-0.729457,-0.258929,0.421871,-0.453863,-0.800754,0.0,-0.682991,-0.239089,-0.333650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,552,601
153,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,532,590
154,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,570,None
155,-0.717189,-0.779855,-0.069186,-0.220165,-0.344018,-0.800754,0.0,-0.682991,-0.239089,-0.210862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,562,None


In [18]:
df_scaled_encoded.to_csv('dataset_scaled.csv')

In [19]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_scaled_encoded)
drop_list_abs =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['abs_nm'].values[row_i] == 'None':
        drop_list_abs.append(row_i)
    
len(drop_list_abs)

5

In [20]:
#Drops the appropriate rows
df_absorbance_scaled_encoded = df_scaled_encoded.drop(drop_list_abs)
#Saves the data for absorbance modelling to CSV
df_absorbance_scaled_encoded.to_csv('dataset_scaled_abs.csv')